<a href="https://colab.research.google.com/github/asilbek-data-scientist/2_modul/blob/main/5_modul_ML_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
import numpy as np
import sklearn # scikit-learn kutubxonasi                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [47]:
# Online dataset joylashgan manzilini ko'rsatamiz
URL = 'https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true'
df = pd.read_csv(URL)

Ma'lumotlarni train va testga ajratamiz.

In [48]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_set.drop('median_house_value', axis=1)
y = train_set['median_house_value'].copy()

X_num = X_train.drop("ocean_proximity", axis=1)

In [49]:
from sklearn.base import BaseEstimator, TransformerMixin
# bizga kearak ustyunlarni indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAddder(BaseEstimator, TransformerMixin):
  def __init__ (self, add_bedrooms_per_room = True):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self, X, y=None):
    return self # bizni funksiyamiz faqat transformer. estimator emas
  def transform(self, X):
    rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
    population_per_household = X[:, population_ix] / X[:, households_ix]
    if self.add_bedrooms_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi.
      bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
    else:
      return np.c_[X, rooms_per_household], population_per_household

In [50]:
from sklearn.pipeline import Pipeline  
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')), # nan qiymatlarni median qiymatlar bn almashtiradi.
    ('attribs_adder', CombinedAttributesAddder(add_bedrooms_per_room=True)), # yangi ustunlarni qo'shyapmiz
    ('std_scaler',  StandardScaler()) # sonlarni ma'lum bir oraliqdagi sonlarga almashitiradi
])


In [51]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num) # sonli ustunlar
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs), #num_attribs uchun num_pipeline ni bajar.
    ('cat', OneHotEncoder(), cat_attribs) # cat_attribs uchun OneHotEncoder ni bajar
])

In [52]:
X_prepared = full_pipeline.fit_transform(X_train)

In [53]:
X_prepared

array([[ 1.27258656, -1.3728112 ,  0.34849025, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813, ...,  0.        ,
         0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.        ,
         0.        ,  0.        ],
       [-1.41489815,  0.99543676,  1.85617335, ...,  0.        ,
         1.        ,  0.        ]])

In [54]:
### Linear Regression

In [55]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()


In [56]:
LR_model.fit(X_prepared, y)

LinearRegression()

In [57]:
test_data = X_train.sample(10)

In [58]:
test_data = X_train.sample(10)
test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
16578,-121.43,37.74,52.0,876.0,170.0,426.0,179.0,3.0865,INLAND
8497,-118.31,33.90,38.0,1400.0,399.0,1131.0,405.0,3.5417,<1H OCEAN
18509,-121.99,36.99,29.0,3119.0,507.0,1476.0,487.0,5.8123,NEAR OCEAN
9198,-119.59,37.39,19.0,3273.0,611.0,1164.0,481.0,3.5446,INLAND
5693,-118.25,34.23,35.0,2839.0,592.0,1413.0,538.0,4.1667,<1H OCEAN
9510,-123.23,39.13,33.0,1176.0,211.0,529.0,217.0,3.8958,<1H OCEAN
1224,-120.57,38.20,13.0,4110.0,847.0,1796.0,706.0,2.6417,INLAND
9964,-122.52,38.53,35.0,1227.0,236.0,548.0,207.0,4.8750,INLAND
16730,-119.93,35.20,29.0,1649.0,342.0,671.0,264.0,3.0602,INLAND
11803,-121.30,38.89,23.0,1750.0,297.0,1012.0,315.0,3.4706,INLAND


In [59]:
test_label = y.loc[test_data.index]
test_label

16578    119800.0
8497     198400.0
18509    281500.0
9198     106500.0
5693     271200.0
9510     144000.0
1224     122300.0
9964     336700.0
16730     69800.0
11803     99300.0
Name: median_house_value, dtype: float64

In [60]:
test_data_prepared = full_pipeline.transform(test_data)
predicted_labels = LR_model.predict(test_data_prepared)

In [61]:
predicted_labels

array([163244.40854017, 236831.03769384, 319832.18752073, 120817.63848824,
       241523.75418792, 222701.06286133,  88438.54639194, 230569.21549582,
       173125.1451369 , 102122.86662386])

In [62]:
pd.DataFrame({'Bashorat': predicted_labels, 'Asl qiymat':test_label})

,Bashorat,Asl qiymat
16578,163244.408540,119800.0
8497,236831.037694,198400.0
18509,319832.187521,281500.0
9198,120817.638488,106500.0
5693,241523.754188,271200.0
9510,222701.062861,144000.0
1224,88438.546392,122300.0
9964,230569.215496,336700.0
16730,173125.145137,69800.0
11803,102122.866624,99300.0


In [63]:
X_test = test_set.drop("median_house_value", axis=1)
X_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...
15362,-117.22,33.36,16.0,3165.0,482.0,1351.0,452.0,4.6050,<1H OCEAN
16623,-120.83,35.36,28.0,4323.0,886.0,1650.0,705.0,2.7266,NEAR OCEAN
18086,-122.05,37.31,25.0,4111.0,538.0,1585.0,568.0,9.2298,<1H OCEAN
2144,-119.76,36.77,36.0,2507.0,466.0,1227.0,474.0,2.7850,INLAND


In [64]:
y_test = test_set["median_house_value"].copy()
y_test

20046     47700.0
3024      45800.0
15663    500001.0
20484    218600.0
9814     278000.0
           ...   
15362    263300.0
16623    266800.0
18086    500001.0
2144      72300.0
3665     151500.0
Name: median_house_value, Length: 4128, dtype: float64

In [65]:
X_test_prepared = full_pipeline.transform(X_test)

In [66]:
y_predicted = LR_model.predict(X_test_prepared)

In [67]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_predicted)
print("MAE=", mae)

MAE= 50898.7395349408


In [68]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_predicted)
print("RSME=", np.sqrt(mse))

RSME= 72701.32600762138


# Random forest


In [69]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

RandomForestRegressor()

In [70]:
y_predicted = RF_model.predict(X_test_prepared)

In [71]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_predicted)
print("RSME=", np.sqrt(mse))

RSME= 49888.56960717251


Cross validation (ma'lumot kam bo'lsa shu usuldan foydalanamiz)

In [72]:
X = df.drop('median_house_value', axis=1)
y = df['median_house_value'].copy()

X_prepared = full_pipeline.transform(X)

In [73]:
from sklearn.model_selection import cross_val_score
mse_scores = cross_val_score(LR_model, X_prepared, y, scoring="neg_mean_squared_error", cv=5 )

In [74]:
def display_scores(scores):
  print("Scores:", scores)
  print("Mean:", scores.mean())
  print("Std.dev:", scores.std())

In [75]:
display_scores(np.sqrt(-mse_scores))

Scores: [73394.92502922 74814.24096819 75431.93119241 76608.78768825
 66196.48128669]
Mean: 73289.27323295093
Std.dev: 3694.7136787223626


In [76]:
scores = cross_val_score(RF_model, X_prepared, y, scoring='neg_mean_squared_error', cv=10) # Random forest uchun ham sinab ko'ramiz
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [97395.5419084  47426.70778157 65124.93785795 56724.21155931
 60713.62479491 60114.72935005 47570.58135598 78772.62017342
 74059.01892946 49416.95721373]
Mean: 63731.89309247922
Std.dev: 15070.901420626016


# Pickle (modelni saqlash)

In [77]:
import pickle
filename = 'RF_model.pkl' # faylga istalgan nom beramiz
with open(filename,"wb") as file:
  pickle.dump(RF_model, file)

In [78]:
with open(filename, "rb") as file: # modelni chaqirish
  model = pickle.load(file)

# Joblib (yordamida saqlash)

In [79]:
import joblib
filename = "LR_model.jbl" # faylga istalgan nom beramiz
joblib.dump(LR_model, filename)

['LR_model.jbl']

In [80]:
model = joblib.load(filename)

In [81]:
scores = cross_val_score(model, X_prepared, y, scoring = "neg_mean_squared_error", cv=5)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [73394.92502922 74814.24096819 75431.93119241 76608.78768825
 66196.48128669]
Mean: 73289.27323295093
Std.dev: 3694.7136787223626
